In [ ]:
import pandas as pd
import plotly.graph_objs as go

In [ ]:
df_n = pd.read_csv("reports/sensitivity_analysis.csv")
df_s = df_n.copy()

df_n.drop(columns=['s' + str(x) for x in range(1,101)], inplace=True)
df_s.drop(columns=['n' + str(x) for x in range(1,101)], inplace=True)

df_n.rename(columns={'n' + str(x): str(x) for x in range(1, 101)}, inplace=True)
df_s.rename(columns={'s' + str(x): str(x) for x in range(1, 101)}, inplace=True)

df_n['species'] = 'N'
df_s['species'] = 'S'

params = ['rii', 'npn', 'mn', 'nps', 'ms', 'sps', 'rrn', 'rrs']
df_n = pd.melt(df_n, id_vars=params+['species'], 
               value_vars=[str(x) for x in range(1,101)], var_name='year', value_name='mean_sn_ratio')
df_s = pd.melt(df_s, id_vars=params+['species'], 
               value_vars=[str(x) for x in range(1,101)], var_name='year', value_name='mean_sn_ratio')


df = pd.concat([df_n, df_s])

print(df.head())
print(len(df))

In [ ]:
no_op = lambda x: 1
for p in params:
    filt_rii = (df['rii'] == 0 ) if p != 'rii' else (df['rii'].apply(no_op))
    filt_npn = (df['npn'] == 51) if p != 'npn' else (df['npn'].apply(no_op))
    filt_mn  = (df['mn']  == 0 ) if p != 'mn'  else (df['mn'].apply(no_op) )
    filt_nps = (df['nps'] == 51) if p != 'nps' else (df['nps'].apply(no_op))
    filt_ms  = (df['ms']  == 0 ) if p != 'ms'  else (df['ms'].apply(no_op) )
    filt_sps = (df['sps'] == 0 ) if p != 'sps' else (df['sps'].apply(no_op))
    filt_rrn = (df['rrn'] == 1 ) if p != 'rrn' else (df['rrn'].apply(no_op))
    filt_rrs = (df['rrs'] == 1 ) if p != 'rrs' else (df['rrs'].apply(no_op))

    f_df = df[filt_rii & filt_npn & filt_mn & filt_nps & filt_ms & filt_sps & filt_rrn & filt_rrs]
    n_df = f_df[f_df['species'] == 'N']
    s_df = f_df[f_df['species'] == 'S']

    layout = go.Layout(title=f'Sensitivity Analysis for {p.upper()}: Mean Native Species Percentage by Year', xaxis={'title':'Year'}, yaxis={'title':'Mean Native Species Percentage'})
    data = [go.Scatter(x=n_df[n_df[p] == val]['year'], 
                       y=n_df[n_df[p] == val]['mean_sn_ratio']*100, 
                       name=f'{p}={val}') for val in n_df[p].unique()
                      ]

    fig = go.Figure(layout=layout, data=data)
    fig.show()
    fig.write_html(f'graphs/sensitivity_analysis_{p}.html')